In [1]:
import random
import time

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchsummary as summary

###Python Random Seed 고정###
SEED = 123456791  # 원하는 seed값을 사용하시면 됩니다.

random.seed(SEED)  # python에서 random 한 부분을 해당 seed값으로 고정합니다.
torch.manual_seed(SEED)  # torch에서 random한 부분을 해당 seed값으로 고정합니다.
torch.cuda.manual_seed(SEED)  # torch의 cuda연산에서 random한 부분을 해당 seed값으로 고정합니다.

###----------------------###


class TypeData(Dataset):
    '''
### Digit일 경우 label로 0을, ###
### Letter일 경우 label로 1을 ###
### return하는 class입니다. ###
사용 예시:
train_data = TypeData(train=True)
test_data = TypeData(train=False)
  '''
    def __init__(self, train):
        super(TypeData, self).__init__()
        self.digit = 10
        self.letter = 46
        self.train = train

        self.data = torchvision.datasets.EMNIST(
            root='./',
            split='bymerge',
            train=self.train,
            transform=transforms.ToTensor(),
            download=True)

    def __getitem__(self, index):
        if self.data[index][1] < self.digit:
            label = 0.
        else:
            label = 1.
        return self.data[index][0], label

    # nn.CrossEntropyLoss()를 사용하는 경우, train 코드에서
    # labels = labels.to(device, dtype=long) 또는,
    # labels = labels.to(device).long() 과 같은 방법으로 data type을 변경하여 사용해 보세요.

    def __len__(self):
        return len(self.data)


### train 또는 test dataset에 대하여, num의 수만큼 subplot을 보여주는 함수입니다.
def image_show(dataset, num):
    fig = plt.figure(figsize=(30, 30))

    for i in range(num):
        plt.subplot(1, num, i + 1)
        plt.imshow(dataset[i][0].squeeze().T)
        plt.title(dataset[i][1])

In [2]:
train_data = torchvision.datasets.EMNIST(root='./',
                                         split='bymerge',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)

test_data = torchvision.datasets.EMNIST(root='./',
                                        split='bymerge',
                                        train=False,
                                        transform=transforms.ToTensor(),
                                        download=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channel = 1
batch_size = 1024
num_epochs = 5
learning_rate = 0.001

In [4]:
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                        batch_size = batch_size,
                                        shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                        batch_size= batch_size,
                                        shuffle=True)

In [5]:
class InceptionBlock(nn.Module):
    def __init__(self, in_nC, n1, in3, n3, in5, n5, npool):
        super(InceptionBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_nC,
                      out_channels=n1,
                      kernel_size=1,
                      stride=1),
            nn.BatchNorm2d(n1),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=in_nC,
                      out_channels=in3,
                      kernel_size=1,
                      stride=1),
            nn.BatchNorm2d(in3),
            nn.ReLU(),
            nn.Conv2d(in_channels=in3,
                      out_channels=n3,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(n3),
            nn.ReLU(),
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=in_nC,
                      out_channels=in5,
                      kernel_size=1,
                      stride=1),
            nn.BatchNorm2d(in5),
            nn.ReLU(),
            nn.Conv2d(in_channels=in5,
                      out_channels=n5,
                      kernel_size=5,
                      stride=1,
                      padding=2),
            nn.BatchNorm2d(n5),
            nn.ReLU(),
        )
        self.pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels=in_nC,
                      out_channels=npool,
                      kernel_size=1,
                      stride=1), nn.BatchNorm2d(npool), nn.ReLU())

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x4 = self.pool(x)
        return torch.cat([x1, x2, x3, x4], dim=1)

In [6]:
class MiniGoogLeNet(nn.Module):
    def __init__(self, num_classes):
        super(MiniGoogLeNet, self).__init__()
        self.chennel_inc = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=1),
            nn.BatchNorm2d(4),
            nn.ReLU(),
        )

        self.a1 = InceptionBlock(4, 8, 8, 16, 4, 8, 4)
        self.b1 = InceptionBlock(36, 12, 8, 12, 4, 16, 8)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.a2 = InceptionBlock(48, 16, 8, 32, 8, 16, 8)
        self.a3 = InceptionBlock(72, 16, 16, 32, 8, 16, 8)
        self.b3 = InceptionBlock(72, 16, 32, 64, 16, 64, 16)
        self.fconv1 = nn.Conv2d(160,num_classes,kernel_size=7)
        self.bn = nn.BatchNorm2d(num_classes)

    def forward(self, x):
        x = self.chennel_inc(x)
        x = self.a1(x)
        x = self.b1(x)
        x = self.maxpool(x)
        x = self.a2(x)
        x = self.maxpool(x)
        x = self.a3(x)
        x = self.b3(x)
        x = self.fconv1(x)
        x = self.bn(x)        
        x = F.softmax(x.reshape(x.size(0), -1))
        return x

In [7]:
model = MiniGoogLeNet(num_classes = 47).to(device)

In [8]:
print(model)

# cf) check the number of parameters
print('{:=^60}'.format("="))
print('{:^60}'.format("model summary"))
print('{:=^60}'.format("="))
for param_tensor in model.state_dict():
    print('%-30s' % param_tensor,
          '{:^30}'.format(str(model.state_dict()[param_tensor].size())))
print('{:=^60}'.format("="))

MiniGoogLeNet(
  (chennel_inc): Sequential(
    (0): Conv2d(1, 4, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (a1): InceptionBlock(
    (conv1): Sequential(
      (0): Conv2d(4, 8, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(4, 8, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (conv5): Sequential(
      (0): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): R

a1.conv3.4.weight                     torch.Size([16])       
a1.conv3.4.bias                       torch.Size([16])       
a1.conv3.4.running_mean               torch.Size([16])       
a1.conv3.4.running_var                torch.Size([16])       
a1.conv3.4.num_batches_tracked         torch.Size([])        
a1.conv5.0.weight                 torch.Size([4, 4, 1, 1])   
a1.conv5.0.bias                       torch.Size([4])        
a1.conv5.1.weight                     torch.Size([4])        
a1.conv5.1.bias                       torch.Size([4])        
a1.conv5.1.running_mean               torch.Size([4])        
a1.conv5.1.running_var                torch.Size([4])        
a1.conv5.1.num_batches_tracked         torch.Size([])        
a1.conv5.3.weight                 torch.Size([8, 4, 5, 5])   
a1.conv5.3.bias                       torch.Size([8])        
a1.conv5.4.weight                     torch.Size([8])        
a1.conv5.4.bias                       torch.Size([8])        
a1.conv5

a3.conv5.3.weight                torch.Size([16, 8, 5, 5])   
a3.conv5.3.bias                       torch.Size([16])       
a3.conv5.4.weight                     torch.Size([16])       
a3.conv5.4.bias                       torch.Size([16])       
a3.conv5.4.running_mean               torch.Size([16])       
a3.conv5.4.running_var                torch.Size([16])       
a3.conv5.4.num_batches_tracked         torch.Size([])        
a3.pool.1.weight                 torch.Size([8, 72, 1, 1])   
a3.pool.1.bias                        torch.Size([8])        
a3.pool.2.weight                      torch.Size([8])        
a3.pool.2.bias                        torch.Size([8])        
a3.pool.2.running_mean                torch.Size([8])        
a3.pool.2.running_var                 torch.Size([8])        
a3.pool.2.num_batches_tracked          torch.Size([])        
b3.conv1.0.weight                torch.Size([16, 72, 1, 1])  
b3.conv1.0.bias                       torch.Size([16])       
b3.conv1

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [10]:
total_step = len(train_loader)
loss_list = []
start = time.time()
lr_update = [3.1,3.02,3,0]
idx = 0
lr_threshold=lr_update[idx]

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()

        outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_list.append(loss)
        end = time.time()
        if (end - start)/60 > 14.9:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            print('time out')
            break
            
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            if loss < lr_threshold:
                idx+=1
                lr_threshold = lr_update[idx]
                optimizer.param_groups[0]['lr']*=0.5
end = time.time()
duration = end - start
print("Training takes {:.2f}minutes".format(duration / 60))

<ipython-input-6-b346c2ba4318>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x.reshape(x.size(0), -1))


Epoch [1/5], Step [10/682], Loss: 3.5080
Epoch [1/5], Step [20/682], Loss: 3.3878
Epoch [1/5], Step [30/682], Loss: 3.3478
Epoch [1/5], Step [40/682], Loss: 3.2973
Epoch [1/5], Step [50/682], Loss: 3.2725
Epoch [1/5], Step [60/682], Loss: 3.2448
Epoch [1/5], Step [70/682], Loss: 3.2558
Epoch [1/5], Step [80/682], Loss: 3.2175
Epoch [1/5], Step [90/682], Loss: 3.1870
Epoch [1/5], Step [100/682], Loss: 3.2044
Epoch [1/5], Step [110/682], Loss: 3.1802
Epoch [1/5], Step [120/682], Loss: 3.1990
Epoch [1/5], Step [130/682], Loss: 3.1626
Epoch [1/5], Step [140/682], Loss: 3.1690
Epoch [1/5], Step [150/682], Loss: 3.1609
Epoch [1/5], Step [160/682], Loss: 3.1478
Epoch [1/5], Step [170/682], Loss: 3.1502
Epoch [1/5], Step [180/682], Loss: 3.1400
Epoch [1/5], Step [190/682], Loss: 3.1319
Epoch [1/5], Step [200/682], Loss: 3.1204
Epoch [1/5], Step [210/682], Loss: 3.1304
Epoch [1/5], Step [220/682], Loss: 3.1169
Epoch [1/5], Step [230/682], Loss: 3.1131
Epoch [1/5], Step [240/682], Loss: 3.1069
E

Epoch [3/5], Step [610/682], Loss: 3.0040
Epoch [3/5], Step [620/682], Loss: 3.0018
Epoch [3/5], Step [630/682], Loss: 3.0051
Epoch [3/5], Step [640/682], Loss: 3.0020
Epoch [3/5], Step [650/682], Loss: 3.0150
Epoch [3/5], Step [660/682], Loss: 2.9958
Epoch [3/5], Step [670/682], Loss: 3.0084
Epoch [3/5], Step [680/682], Loss: 3.0051
Epoch [4/5], Step [10/682], Loss: 2.9954
Epoch [4/5], Step [20/682], Loss: 3.0017
Epoch [4/5], Step [30/682], Loss: 3.0060
Epoch [4/5], Step [40/682], Loss: 2.9896
Epoch [4/5], Step [50/682], Loss: 2.9946
Epoch [4/5], Step [60/682], Loss: 3.0004
Epoch [4/5], Step [70/682], Loss: 3.0132
Epoch [4/5], Step [80/682], Loss: 3.0076
Epoch [4/5], Step [90/682], Loss: 3.0004
Epoch [4/5], Step [100/682], Loss: 3.0221
Epoch [4/5], Step [110/682], Loss: 3.0053
Epoch [4/5], Step [120/682], Loss: 2.9897
Epoch [4/5], Step [130/682], Loss: 2.9922
Epoch [4/5], Step [140/682], Loss: 2.9997
Epoch [4/5], Step [150/682], Loss: 3.0166
Epoch [4/5], Step [160/682], Loss: 3.0173
E

In [11]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} test images {}%'.format(
        len(test_loader) * batch_size, 100 * correct / total))


<ipython-input-6-b346c2ba4318>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x.reshape(x.size(0), -1))


Accuracy of the network on the 116736 test images 90.62266275801002%


In [12]:
torch.save(model.state_dict(), './model_bymerge.pth')